In [1]:
import os
import glob
import numpy as np
import pandas as pd
from functions import split_preds, metrics, measure_csv_df
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
os.chdir("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/files/Datasets")

Data Taken from http://www.football-data.co.uk/englandm.php

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
#combine all files in the list
master_sheet = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
master_sheet.to_csv( "master_sheet.csv", index=False, encoding='utf-8-sig')

Reduce table to using columns needed for the model

Columns Required:

Date, Time HomeTeam, AwayTeam, FullTimeHG, FullTimeAG, FullTimeRresults, HalfTimeHG, HalfTimeAG, HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

Columns To Consider:

Attendance, Referee

In [4]:
subset_master = master_sheet[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
subset_master = subset_master.fillna(0)
# 0 = Home Wins, 1 = Draw, 2 = Away Wins
subset_master.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)

In [5]:
subset_master.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0


In [6]:
msk = np.random.rand(len(subset_master)) < 0.75

train_df = subset_master[msk]
test_df = subset_master[~msk]

X_train = train_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG"])
y_train = train_df[['FTR']] 
X_test = test_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG", ])
y_test = test_df[['FTR']]

In [7]:
clf = RandomForestClassifier(n_estimators = 2000, max_depth=3, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)
split_preds(subset_master, y_pred)

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,...,4.0,12.0,6.0,14.0,10.0,0.0,0.0,0.577811,0.213650,0.208539
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,...,1.0,2.0,3.0,10.0,18.0,0.0,0.0,0.407653,0.236707,0.355640
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,...,6.0,6.0,3.0,17.0,12.0,0.0,0.0,0.591106,0.260716,0.148178
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,...,4.0,2.0,5.0,9.0,11.0,0.0,1.0,0.560516,0.220598,0.218885
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,...,7.0,3.0,2.0,9.0,16.0,1.0,0.0,0.567132,0.223534,0.209334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,...,2.0,5.0,6.0,7.0,6.0,0.0,0.0,0.275281,0.242022,0.482697
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,...,4.0,8.0,4.0,13.0,8.0,0.0,0.0,0.612977,0.211300,0.175722
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,...,1.0,2.0,3.0,9.0,9.0,0.0,0.0,0.232086,0.245853,0.522061
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,...,4.0,3.0,9.0,13.0,4.0,0.0,0.0,0.463763,0.248588,0.287649


Identifying how often Predicted Results decided the outcome (win, draw, away) against the actual results)

In [8]:
metrics(subset_master)

False    55.871645
True     44.128355
Name: Comparison, dtype: float64
0    45.295061
2    30.153475
1    24.551464
Name: FTR, dtype: float64
0    92.685188
2     7.314812
Name: PredResult, dtype: float64


In [9]:
subset_master

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc,PredResult,Comparison
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,...,6.0,14.0,10.0,0.0,0.0,0.577811,0.213650,0.208539,0,False
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,...,3.0,10.0,18.0,0.0,0.0,0.407653,0.236707,0.355640,0,False
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,...,3.0,17.0,12.0,0.0,0.0,0.591106,0.260716,0.148178,0,False
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,...,5.0,9.0,11.0,0.0,1.0,0.560516,0.220598,0.218885,0,False
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,...,2.0,9.0,16.0,1.0,0.0,0.567132,0.223534,0.209334,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,...,6.0,7.0,6.0,0.0,0.0,0.275281,0.242022,0.482697,2,False
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,...,4.0,13.0,8.0,0.0,0.0,0.612977,0.211300,0.175722,0,True
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,...,3.0,9.0,9.0,0.0,0.0,0.232086,0.245853,0.522061,2,False
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,...,9.0,13.0,4.0,0.0,0.0,0.463763,0.248588,0.287649,0,True


Columns Required for predicting:

HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

In [22]:
current_season = pd.read_csv("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/2020-2021.csv")
current_season_subset = current_season[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
current_season_subset_pred = current_season_subset.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG"])
current_season_subset_pred.head()

,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,5,13,2,6,2,3,12,12,0,0
1,5,9,3,5,7,3,14,11,0,0
2,22,6,6,3,9,0,9,6,0,0
3,15,15,3,2,8,7,13,7,0,0
4,7,13,1,7,2,5,12,9,0,0


In [26]:
y_pred = clf.predict_proba(current_season_subset_pred)
y_pred = pd.DataFrame(y_pred)

In [29]:
current_season_subset = measure_csv_df(current_season_subset,clf)
current_season_subset.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)
current_season_subset

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,12/09/2020,12:30,Fulham,Arsenal,0,3,2,0,1,5,...,6,2,3,12,12,0,0,0.209656,0.239536,0.550808
1,12/09/2020,15:00,Crystal Palace,Southampton,1,0,0,1,0,5,...,5,7,3,14,11,0,0,0.308002,0.269363,0.422635
2,12/09/2020,17:30,Liverpool,Leeds,4,3,0,3,2,22,...,3,9,0,9,6,0,0,0.586747,0.227029,0.186224
3,12/09/2020,20:00,West Ham,Newcastle,0,2,2,0,0,15,...,2,8,7,13,7,0,0,0.440358,0.286774,0.272868
4,13/09/2020,14:00,West Brom,Leicester,0,3,2,0,0,7,...,7,2,5,12,9,0,0,0.223009,0.233947,0.543044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,07/02/2021,12:00,Tottenham,West Brom,2,0,0,0,0,13,...,1,8,0,16,12,0,0,0.645299,0.225844,0.128858
221,07/02/2021,14:00,Wolves,Leicester,0,0,1,0,0,13,...,3,8,2,11,15,0,0,0.343122,0.302763,0.354115
222,07/02/2021,16:30,Liverpool,Man City,1,4,2,0,0,8,...,5,6,1,13,8,0,0,0.319791,0.260483,0.419726
223,07/02/2021,19:15,Sheffield United,Chelsea,1,2,2,0,1,8,...,3,1,8,8,8,0,0,0.416117,0.305466,0.278417
